In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
subsample_size = 500  # subsample subset of data for faster demo, try setting this to much larger values
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


In [4]:
label = 'class'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count        500
unique         2
top        <=50K
freq         365
Name: class, dtype: object


In [5]:
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15891.82 MB
	Train

In [6]:
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

Loaded data from: https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv | Columns = 15 / 15 | Rows = 9769 -> 9769


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States


In [7]:
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: accuracy on test data: 0.8397993653393387
Evaluations on test data:
{
    "accuracy": 0.8397993653393387,
    "balanced_accuracy": 0.7437076677780596,
    "mcc": 0.5295565206264157,
    "f1": 0.6242496998799519,
    "precision": 0.7038440714672441,
    "recall": 0.5608283002588438
}


Predictions:  
 0        <=50K
1        <=50K
2         >50K
3        <=50K
4        <=50K
         ...  
9764     <=50K
9765     <=50K
9766     <=50K
9767     <=50K
9768     <=50K
Name: class, Length: 9769, dtype: object


In [8]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.842666,0.85,0.092894,0.006926,0.241356,0.092894,0.006926,0.241356,1,True,11
1,RandomForestGini,0.841335,0.84,0.217121,0.108091,0.672280,0.217121,0.108091,0.672280,1,True,5
2,RandomForestEntr,0.840721,0.83,0.216123,0.107722,0.522934,0.216123,0.107722,0.522934,1,True,6
3,LightGBM,0.839799,0.85,0.046232,0.012825,0.180073,0.046232,0.012825,0.180073,1,True,4
4,WeightedEnsemble_L2,0.839799,0.85,0.048691,0.013477,0.559978,0.002458,0.000652,0.379905,2,True,14
5,LightGBMXT,0.839390,0.83,0.031727,0.012664,1.011925,0.031727,0.012664,1.011925,1,True,3
6,CatBoost,0.837957,0.84,0.021698,0.011843,0.854296,0.021698,0.011843,0.854296,1,True,7
7,ExtraTreesEntr,0.834783,0.82,0.318504,0.107988,0.525438,0.318504,0.107988,0.525438,1,True,9
8,ExtraTreesGini,0.834476,0.82,0.320222,0.107936,0.523781,0.320222,0.107936,0.523781,1,True,8
9,NeuralNetFastAI,0.831405,0.83,0.206189,0.037177,2.395914,0.206189,0.037177,2.395914,1,True,10


# Maximizing predictive performance¶

In [10]:
time_limit = 60  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20210502_142728/"
Presets specified: ['best_quality']
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20210502_142728/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [' >50K', ' <=50K']
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 =  >50K, class 0 =  <=50K
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive ( >50K) vs negative ( <=50K) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Us

In [11]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost_BAG_L1,0.902783,0.887489,0.096827,0.046946,5.318545,0.096827,0.046946,5.318545,1,True,7
1,LightGBMXT_BAG_L1,0.900161,0.881380,0.378683,0.061646,0.994690,0.378683,0.061646,0.994690,1,True,3
2,WeightedEnsemble_L2,0.896811,0.900944,8.610355,1.144391,36.734011,0.005073,0.001215,1.353384,2,True,14
3,LightGBM_BAG_L1,0.892347,0.866991,0.173242,0.060415,0.984328,0.173242,0.060415,0.984328,1,True,4
4,XGBoost_BAG_L1,0.891681,0.866575,0.458843,0.031060,0.809559,0.458843,0.031060,0.809559,1,True,11
5,RandomForestEntr_BAG_L1,0.888119,0.886301,0.217607,0.095736,0.623113,0.217607,0.095736,0.623113,1,True,6
6,RandomForestGini_BAG_L1,0.886598,0.884698,0.217541,0.096282,0.522590,0.217541,0.096282,0.522590,1,True,5
7,NeuralNetFastAI_BAG_L1,0.882267,0.858488,1.025242,0.145782,5.097872,1.025242,0.145782,5.097872,1,True,10
8,ExtraTreesGini_BAG_L1,0.881065,0.892927,0.318996,0.096380,0.522779,0.318996,0.096380,0.522779,1,True,8
9,ExtraTreesEntr_BAG_L1,0.880851,0.893912,0.319165,0.096801,0.523841,0.319165,0.096801,0.523841,1,True,9


# Regression (predicting numeric table columns)

In [12]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

Summary of age variable: 
 count    500.00000
mean      39.65200
std       13.52393
min       17.00000
25%       29.00000
50%       38.00000
75%       49.00000
max       85.00000
Name: age, dtype: float64


In [13]:
predictor_age = TabularPredictor(label=age_column, path="agModels-predictAge").fit(train_data, time_limit=60)
performance = predictor_age.evaluate(test_data)

Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge/"
AutoGluon Version:  0.2.0
Train Data Rows:    500
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (85, 17, 39.652, 13.52393)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    15670.06 MB
	Train Data (Original)  Memory Usage: 0.32 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...

In [14]:
predictor_age.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-10.459282,-11.331723,2.080282,0.368140,10.737966,0.005555,0.000587,0.523062,2,True,12
1,ExtraTreesMSE,-10.691115,-11.480752,0.320128,0.108661,0.517839,0.320128,0.108661,0.517839,1,True,7
2,RandomForestMSE,-10.746518,-11.602848,0.319801,0.108607,0.618975,0.319801,0.108607,0.618975,1,True,5
3,LightGBMXT,-10.753344,-11.814712,0.111668,0.014050,0.218307,0.111668,0.014050,0.218307,1,True,3
4,CatBoost,-10.800412,-11.744795,0.022473,0.010796,0.591322,0.022473,0.010796,0.591322,1,True,6
5,LightGBM,-10.972156,-11.929546,0.055763,0.014220,0.195404,0.055763,0.014220,0.195404,1,True,4
6,XGBoost,-11.121008,-12.174270,0.110365,0.007207,0.214001,0.110365,0.007207,0.214001,1,True,9
7,NeuralNetMXNet,-11.560166,-12.632272,1.246197,0.118062,8.077363,1.246197,0.118062,8.077363,1,True,10
8,LightGBMLarge,-11.598649,-12.167606,0.142061,0.014685,0.413190,0.142061,0.014685,0.413190,1,True,11
9,KNeighborsUnif,-14.902058,-15.686937,0.103323,0.102887,0.003339,0.103323,0.102887,0.003339,1,True,1
